In [1]:
from tqdm import tqdm

# Custom
import models.resnet as resnet
import models.lossnet as lossnet
from config import *
from data.fst_data import train_data, test_data

import timm
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

import datetime
# Python
import os
import random
import time
# Torch
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data.sampler import SubsetRandomSampler

# Torchvison
import torchvision.transforms as T
import torchvision.models as models

import torch
from torchvision.models import resnet50
from torchvision.models.feature_extraction import get_graph_node_names
from torchvision.models.feature_extraction import create_feature_extractor
from torchvision.models import ResNet50_Weights

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

print(device)

print("쿠다 가능 :{}".format(torch.cuda.is_available()))

import warnings
warnings.filterwarnings('ignore')



cpu
쿠다 가능 :False


In [3]:
def LossPredLoss(input, target, margin=1.0, reduction='mean'):
    assert len(input) % 2 == 0, 'the batch size is not even.'
    assert input.shape == input.flip(0).shape
    
    input = (input - input.flip(0))[:len(input)//2] # [l_1 - l_2B, l_2 - l_2B-1, ... , l_B - l_B+1], where batch_size = 2B
    target = (target - target.flip(0))[:len(target)//2]
    target = target.detach()

    one = 2 * torch.sign(torch.clamp(target, min=0)) - 1 # 1 operation which is defined by the authors
    
    if reduction == 'mean':
        loss = torch.sum(torch.clamp(margin - one * input, min=0))
        loss = loss / input.size(0) # Note that the size of input is already halved
    elif reduction == 'none':
        loss = torch.clamp(margin - one * input, min=0)
    else:
        NotImplementedError()
    
    return loss

# Train Utils
iters = 0


def get_uncertainty(models, extractor, unlabeled_loader):
    models['backbone'].eval()
    models['module'].eval()
    uncertainty = torch.tensor([]).to(device)

    with torch.no_grad():
        for (inputs, labels) in unlabeled_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            scores = models['backbone'](inputs)
            
            features = extractor
            pred_loss = models['module'](features) # pred_loss = criterion(scores, labels) # ground truth loss
            pred_loss = pred_loss.view(pred_loss.size(0))

            uncertainty = torch.cat((uncertainty, pred_loss), 0)
    
    return uncertainty.cpu()


def test(models, extractor, dataloaders, mode='val'):
    metrics = {}
    preds, labels = [], []
    assert mode == 'val' or mode == 'test'
    models['backbone'].eval()
    models['module'].eval()
    

    with torch.no_grad():
        for data in dataloaders[mode]:
            inputs = data[0].to(device)
            label = data[1].to(device)
            # inputs = inputs.to(device)
            # label = label.to(device)

            scores = models['backbone'](inputs)
            # _, preds = torch.max(scores.data, 1)
            labels.extend(label.detach().tolist())
            preds.extend(scores.argmax(axis=1).detach().tolist())
        print(f" labels = {labels}")
        print(f" preds = {preds}")
        
    metrics['accuracy'] = accuracy_score(y_pred=preds, y_true=labels)
    metrics['f1_score'] = f1_score(y_pred=preds, y_true=labels, average='weighted')
    metrics['precision'] = precision_score(y_pred=preds, y_true=labels, average='weighted')
    metrics['recall'] = recall_score(y_pred=preds, y_true=labels, average='weighted')

            # print("labels " , labels)
            # print("preds " , preds)
    return metrics

#
def train(models, extractor, criterion, optimizers, schedulers, dataloaders, num_epochs, epoch_loss):
    print('>> Train a Model.')
    best_acc = 0.
    checkpoint_dir = os.path.join('./cifar10', 'train', 'weights')
    if not os.path.exists(checkpoint_dir):
        os.makedirs(checkpoint_dir)
    
    for epoch in tqdm(range(num_epochs)):
            
        train_epoch(models, extractor, criterion, optimizers, dataloaders, epoch, epoch_loss)
        schedulers['backbone'].step()
        schedulers['module'].step()

    print('>> Finished.')
    
def train_epoch(models, extractor, criterion, optimizers, dataloaders, epoch, epoch_loss):
    models['backbone'].train()
    models['module'].train()
    global iters

    for data in tqdm(dataloaders['train'], leave=False, total=len(dataloaders['train'])):
        inputs = data[0].to(device)
        labels = data[1].to(device)
        img_name = data[2]
        
        iters += 1

        optimizers['backbone'].zero_grad()
        optimizers['module'].zero_grad()

        scores = models['backbone'](inputs)
        target_loss = criterion(scores, labels)
        
        features_dic = extractor(inputs)
        features = [features_dic['layer1'], features_dic['layer2'], features_dic['layer3'], features_dic['layer4']]
        
        # print(f" feature_length = {len(features)}")
        
        # print(features)
        
        if epoch > epoch_loss:
            # After 120 epochs, stop the gradient from the loss prediction module propagated to the target model.
            features[0] = features[0].detach()
            features[1] = features[1].detach()
            features[2] = features[2].detach()
            features[3] = features[3].detach()
        
            
        pred_loss = models['module'](features)
        pred_loss = pred_loss.view(pred_loss.size(0))


        m_backbone_loss = torch.sum(target_loss) / target_loss.size(0)
        m_module_loss   = LossPredLoss(pred_loss, target_loss, margin=MARGIN)
        loss            = m_backbone_loss + WEIGHT * m_module_loss

        loss.backward()
        optimizers['backbone'].step()
        optimizers['module'].step()



In [6]:
from torchvision.models import resnet50

train_loader = DataLoader(train_data, batch_size=BATCH, 
                                  num_workers = 1, pin_memory=True,  drop_last = True)
test_loader  = DataLoader(test_data,num_workers = 1, batch_size=1)

dataloaders  = {'train': train_loader, 'test': test_loader}

res101 = resnet50(weights=None).to(device)
# train_nodes, eval_nodes = get_graph_node_names(res101)
# print(train_nodes)
# print()
# print(eval_nodes)

extractor = create_feature_extractor(
    res101,
    return_nodes=[
        "layer1",
        "layer2",
        "layer3",
        "layer4",
        # "layer5"
    ]
)

# features = extractor(img)   
# print(len(features))

# print(f"resnet summary : {summary_(resnet18, (3, 32, 32))}")

# resnet18 = timm.create_model("resnet50", num_classes = 4, pretrained=True).to(device)
loss_module = lossnet.LossNet().to(device)
models      = {'backbone': res101 , 'module': loss_module}

torch.backends.cudnn.benchmark = False

# Active learning cycles
for cycle in range(CYCLES):
    # Loss, criterion and scheduler (re)initialization
    criterion      = nn.CrossEntropyLoss(reduction='none')
    optim_backbone = optim.SGD(models['backbone'].parameters(), lr=LR, 
                            momentum=MOMENTUM, weight_decay=WDECAY)
    optim_module   = optim.SGD(models['module'].parameters(), lr=LR, 
                            momentum=MOMENTUM, weight_decay=WDECAY)
    sched_backbone = lr_scheduler.MultiStepLR(optim_backbone, milestones=MILESTONES)
    sched_module   = lr_scheduler.MultiStepLR(optim_module, milestones=MILESTONES)

    optimizers = {'backbone': optim_backbone, 'module': optim_module}
    schedulers = {'backbone': sched_backbone, 'module': sched_module}

    # Training and test
    train(models, extractor, criterion, optimizers, schedulers, dataloaders, EPOCH, EPOCHL)
    metrics = test(models, extractor, dataloaders, mode='test')
    
    print(f"accuracy : {metrics['accuracy']:.4f}")
    print(f"f1 score : {metrics['f1_score']:.4f}")
    print(f"precision : {metrics['precision']:.4f}")
    print(f"recall : {metrics['recall']:.4f}")
    print('Trial {}/{} || Cycle {}/{} || Label set size {}'.format(trial+1, TRIALS, cycle+1, CYCLES, len(labeled_set)))
    
    
    acc_list.append(metrics['accuracy'])
    f1_list.append(metrics['f1_score'])
    ##
    #  Update the labeled dataset via loss prediction-based uncertainty measurement

    # Randomly sample 10000 unlabeled data points
    random.shuffle(unlabeled_set)
    subset = unlabeled_set[:SUBSET]

    # Create unlabeled dataloader for the unlabeled subset
    unlabeled_loader = DataLoader(unlabeled_data, batch_size=BATCH, 
                                    sampler=SubsetSequentialSampler(subset), # more convenient if we maintain the order of subset
                                    num_workers = 1, pin_memory=True,  drop_last = True)

    # Measure uncertainty of each data points in the subset
    uncertainty = get_uncertainty(models, extractor, unlabeled_loader)

    # Index in ascending order
    arg = np.argsort(uncertainty)
    
    # Update the labeled dataset and the unlabeled dataset, respectively
    labeled_set += list(torch.tensor(subset)[arg][-ADDENDUM:].numpy())
    unlabeled_set = list(torch.tensor(subset)[arg][:-ADDENDUM].numpy()) + unlabeled_set[SUBSET:]

    # Create a new dataloader for the updated labeled dataset
    
    train_subset = SubsetRandomSampler(labeled_set)
    
    dataloaders['train'] = DataLoader(train_data, batch_size=BATCH, 
                                        sampler=train_subset, 
                                        num_workers = 1, pin_memory=True,  drop_last = True)


['x', 'conv1', 'bn1', 'relu', 'maxpool', 'layer1.0.conv1', 'layer1.0.bn1', 'layer1.0.relu', 'layer1.0.conv2', 'layer1.0.bn2', 'layer1.0.relu_1', 'layer1.0.conv3', 'layer1.0.bn3', 'layer1.0.downsample.0', 'layer1.0.downsample.1', 'layer1.0.add', 'layer1.0.relu_2', 'layer1.1.conv1', 'layer1.1.bn1', 'layer1.1.relu', 'layer1.1.conv2', 'layer1.1.bn2', 'layer1.1.relu_1', 'layer1.1.conv3', 'layer1.1.bn3', 'layer1.1.add', 'layer1.1.relu_2', 'layer1.2.conv1', 'layer1.2.bn1', 'layer1.2.relu', 'layer1.2.conv2', 'layer1.2.bn2', 'layer1.2.relu_1', 'layer1.2.conv3', 'layer1.2.bn3', 'layer1.2.add', 'layer1.2.relu_2', 'layer2.0.conv1', 'layer2.0.bn1', 'layer2.0.relu', 'layer2.0.conv2', 'layer2.0.bn2', 'layer2.0.relu_1', 'layer2.0.conv3', 'layer2.0.bn3', 'layer2.0.downsample.0', 'layer2.0.downsample.1', 'layer2.0.add', 'layer2.0.relu_2', 'layer2.1.conv1', 'layer2.1.bn1', 'layer2.1.relu', 'layer2.1.conv2', 'layer2.1.bn2', 'layer2.1.relu_1', 'layer2.1.conv3', 'layer2.1.bn3', 'layer2.1.add', 'layer2.1.rel

ValueError: node: 'layer5' is not present in model. Hint: use `get_graph_node_names` to make sure the `return_nodes` you specified are present. It may even be that you need to specify `train_return_nodes` and `eval_return_nodes` separately.

In [ ]:
print(f"accuracy : {metrics['accuracy']:.4f}")
print(f"f1 score : {metrics['f1_score']:.4f}")
print(f"precision : {metrics['precision']:.4f}")
print(f"recall : {metrics['recall']:.4f}")
print('Trial {}/{} || Cycle {}/{} || Label set size {}'.format(trial+1, TRIALS, cycle+1, CYCLES, len(labeled_set)))


In [ ]:
exit()

In [ ]:
from tqdm import tqdm

# Custom
import models.resnet as resnet
import models.lossnet as lossnet
from config import *
from data.fst_data import train_data, test_data

import timm
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

import datetime
# Python
import os
import random
import time
# Torch
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr_scheduler
from torch.utils.data.sampler import SubsetRandomSampler

# Torchvison
import torchvision.transforms as T
import torchvision.models as models

model1 = timm.create_model('resnet50', num_classes = 4, pretrained=True, features_only = True, out_indices = (1,2,3,4))
model2 = timm.create_model('resnet50', num_classes = 4, pretrained=True)

In [ ]:
from PIL import Image
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torchvision.transforms as T
import timm
from pathlib import Path


img = Image.open("/Users/ibyeong-gwon/Desktop/Git/DeepLearning/211201-AT6T31EH-M232_inpaint_1 ADR_W1.6_H1.7_FS.jpg")
train_transform = T.Compose([
    # transforms.Resize((32, 32)),
    # transforms.Resize((224, 224)),
    transforms.Resize((360, 360)),
    T.RandomHorizontalFlip(),
    # T.RandomCrop(size=360, padding=4),
    
    T.ToTensor(),
    
    # T.Normalize([0.4914, 0.4822, 0.4465], [0.2023, 0.1994, 0.2010]) # T.Normalize((0.5071, 0.4867, 0.4408), (0.2675, 0.2565, 0.2761)) # CIFAR-100
])

img = train_transform(img)
img = img.unsqueeze(0)


In [ ]:
import torch.nn as nn
import torch.nn.functional as F
img = Image.open("/Users/ibyeong-gwon/Desktop/Git/DeepLearning/211201-AT6T31EH-M232_inpaint_1 ADR_W1.6_H1.7_FS.jpg")
img = train_transform(img)
img = img.unsqueeze(0)

out4 = model1(img)[3]
out = F.avg_pool2d(out4, 4)
        # print(out.shape)
out = out.view(out.size(0), -1)
print("out shape = ",out.shape)

linear = nn.Linear(18432, 4)

out = linear(out)

print(out)

print(model2(img))

In [ ]:
model2 = timm.create_model('resnet50', num_classes = 4, pretrained=True)
model2.forward_features(img).size()

In [ ]:
(model1(img)[3])

In [ ]:
import torch
from torchvision.models import resnet50
from torchvision.models.feature_extraction import get_graph_node_names
from torchvision.models.feature_extraction import create_feature_extractor

res101 = models.resnet50(pretrained=True)

extractor = create_feature_extractor(
    res101,
    return_nodes=[
        "layer1",
        "layer2",
        "layer3",
        "layer4",
    ]
)

features = extractor(img)   
print(len(features))